In [1]:
# Adding src module to path
import sys
import os
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
# general imports
from datetime import datetime
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

import src.utils as utils
from src.ansatz import RBM


ModuleNotFoundError: No module named 'src.ansatz'

## Testing hidden node accuracy

Training several RBMs with increasing number of visible nodes. For each RBM we test how increasing the number of hidden nodes affects accuracy.

In [ ]:

#Initializing starting values 
n_vis = 2
vis_inc = 2
total_vis = 4


n_hid_start = 4
n_hid = n_hid_start
hid_inc = 2
total_hid = 5

energy_list = []
n_hid_list =[]

seed = 42


username = os.getlogin()
desktop_path = f"{os.path.expanduser(os.getenv('USERPROFILE'))}\\Desktop"

date_format = '%Y.%m.%d;%H%M'

path = f"{desktop_path}\\{datetime.now().strftime(date_format)}-RBMplots"
os.makedirs(path)

for i in tqdm(range(total_vis), position=0, desc=f"Total", leave=False, colour='green'):
    
    #Finding true ground state energy and displaying it
    np.random.seed(seed)
    
    hamiltonian = utils.random_hamiltonian(2**n_vis)
    eig,_ = np.linalg.eigh(hamiltonian)
    E_truth = np.min(eig)
    #print(f"Energy truth: {E_truth}")
    
    for j in tqdm(range(total_hid), position=1, desc=f"Training", leave=False, colour='red'):
        np.random.seed(seed)
        
        hamiltonian = utils.random_hamiltonian(2**n_vis)

        rbm = RBM(visible_size=n_vis, hidden_size=n_hid, hamiltonian=hamiltonian)
        energy_list.append(rbm.train(iter=1000, lr=0.01))
        n_hid_list.append(n_hid)
        n_hid *= hid_inc
    
    # Print loop    
    labels = [f"Hidden = {i}" for i in n_hid_list]
    labels.append("True energy")
    
    # plt.ioff() # uncomment to stop plotting in notebook
    plt.figure(i)
    
    
    for energy in energy_list:
        plt.plot(energy)
        plt.xlabel('Gradient steps')
        plt.ylabel('Energy')

    plt.title(f"Training of RBM with {n_vis} visible nodes")
    plt.axhline(y=E_truth, color='k', linestyle='--')
    plt.legend(labels)

    # Save figure as pdf
    plt.savefig(f"{path}/{n_vis}nodes.pdf")
    
    n_vis += vis_inc
    n_hid = n_hid_start
    energy_list.clear()
    n_hid_list.clear()
    labels.clear()

